In [2]:
from tinydb import TinyDB, where

In [3]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Load environment variables from .env file
load_dotenv()

# Read Neo4j credentials from environment variables
neo4j_username = os.getenv('NEO4J_USERNAME')
neo4j_password = os.getenv('NEO4J_PASSWORD')
neo4j_uri = os.getenv('NEO4J_URI')

# Create a Neo4j driver instance
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))

# Verify the connection

def verify_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            if result.single()[0] == 1:
                print("Connection to Neo4j established successfully.")
            else:
                print("Failed to establish connection to Neo4j.")
    except Exception as e:
        print(f"An error occurred: {e}")

verify_connection(driver)



Connection to Neo4j established successfully.


In [4]:
import textwrap

def recursive_text_splitter(document, max_chunk_size):
    # Use textwrap to initially split the text into lines of max_chunk_size
    chunks = textwrap.wrap(document, width=max_chunk_size)
    
    final_chunks = []
    
    for chunk in chunks:
        if len(chunk) > max_chunk_size:
            # If a chunk is still larger than max_chunk_size, split it further
            final_chunks.extend(recursive_text_splitter(chunk, max_chunk_size))
        else:
            final_chunks.append(chunk)
    
    return final_chunks

In [5]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [6]:
import os
from pinecone import Pinecone

# Load Pinecone API key from environment variables
pinecone_api_key = os.getenv('PINECONE_API_KEY')
print(pinecone_api_key)

# Initialize Pinecone client
pc = Pinecone(api_key=pinecone_api_key)

# Create a Pinecone index
index_name = "scotus-improved"
pindex = pc.Index(index_name)


40dcba58-adbd-4509-9323-f47d8cf22799


In [7]:
def save_opinion_embedding(case_id, opinion_text):
    chunks = recursive_text_splitter(opinion_text, 512)
    
    # Create an index if it doesn't exist  
    for chunk in chunks:
        chunk_id = str(case_id) + "_" + str(chunks.index(chunk))
        embedding = get_embedding(chunk)        
        metadata = {
            "case_id": str(case_id),
            "chunk": chunk
        }
        
        for key, value in metadata.items():
            if value is None:
                metadata[key] = "null" 

        pindex.upsert([(chunk_id, embedding, metadata)])

In [8]:
import re
def escape_neo4j_string(value):
    if isinstance(value, str):
        # Escape single quotes, double quotes, backslashes, and special characters
        value = re.sub(r"(['\"\\])", r"\\\1", value)
        value = re.sub(r"([{}:])", r"\\\1", value)
        # Replace spaces with underscores in relationship types
        value = re.sub(r"\s+", "_", value)
    return value

class Node:
    def __init__(self, id, label, properties):
        self.id = escape_neo4j_string(id)
        self.label = self.sanitize_label(label)
        self.properties = {k: v if isinstance(v, str) else str(v) for k, v in properties.items()}

    def sanitize_label(self, label):
        # Remove any dots and replace with underscores
        return escape_neo4j_string(str(label).replace('.', '_'))

    def create_node_query(self):
        props = ', '.join([f"{key}: ${key}" for key in self.properties.keys()])
        if 'name' in self.properties:
            return f"""
            MERGE (n:{self.label} {{id: '{self.id}'}})
            ON CREATE SET n += $properties
            WITH n
            MATCH (existing:{self.label} {{name: $name}})
            WHERE existing.id <> n.id
            WITH n, existing
            SET n += properties(existing)
            RETURN n
            """
        else:
            return f"MERGE (n:{self.label} {{id: '{self.id}', {props}}})"

    def create_node(self):
        if not self.id or not self.label or any(value is None for value in self.properties.values()):
            return
        query = self.create_node_query()
        escaped_props = {k: v.replace("'", "\\'") if isinstance(v, str) else v for k, v in self.properties.items()}
        with driver.session() as session:
            session.run(query, properties=escaped_props, **escaped_props)


class Edge:
    def __init__(self, from_node_id, to_node_id, relationship_type, properties, increment_property):
        self.from_node_id = escape_neo4j_string(from_node_id)
        self.to_node_id = escape_neo4j_string(to_node_id)
        self.relationship_type = escape_neo4j_string(relationship_type)
        self.properties = {k: escape_neo4j_string(v) if isinstance(v, str) else v for k, v in properties.items()}
        self.increment_property = increment_property

    def create_edge_query(self):
        props = ', '.join([f"{key}: ${key}" for key in self.properties.keys()])
        return (f"MATCH (a {{id: '{self.from_node_id}'}}), (b {{id: '{self.to_node_id}'}}) "
                f"MERGE (a)-[r:{self.relationship_type} {{{props}}}]->(b) "
                f"ON CREATE SET r.{self.increment_property} = 1 "
                f"ON MATCH SET r.{self.increment_property} = r.{self.increment_property} + 1")

    def create_edge(self):
        query = self.create_edge_query()
        with driver.session() as session:
            session.run(query, **self.properties)



In [9]:
from datetime import datetime

class Entity:
    def __init__(self, text, label):
        self.text = text
        self.label = label

class Relation:
    def __init__(self, label, head, tail):
        self.label = label
        self.head = head
        self.tail = tail

class Citation:
    def __init__(self, data):
        self.volume = data.get('volume')
        self.page = data.get('page')
        self.year = data.get('year')

    def __str__(self):
        return f"{self.volume} U.S. {self.page} ({self.year})"

class Advocate:
    def __init__(self, data):
        advocate_data = data.get('advocate', {})
        self.name = advocate_data.get('name') if advocate_data else None
        self.description = data.get('advocate_description')

    def __str__(self):
        return f"{self.name}: {self.description}"

class Decision:
    def __init__(self, data):
        self.description = data.get('description')
        self.winning_party = data.get('winning_party')
        self.decision_type = data.get('decision_type')
        self.votes = [Vote(v) for v in data.get('votes', []) if v]

    def __str__(self):
        return f"{self.description} - Winner: {self.winning_party}"

class Vote:
    def __init__(self, data):
        self.member = Justice(data.get('member', {}))
        self.vote = data.get('vote')
        self.opinion_type = data.get('opinion_type')
        self.href = data.get('href')

    def __str__(self):
        return f"{self.member.name}: {self.vote}"

class Justice:
    def __init__(self, data):
        self.id = data.get('ID')
        self.name = data.get('name')
        self.roles = [Role(r) for r in data.get('roles', []) if r]

    def __str__(self):
        return self.name

class Role:
    def __init__(self, data):
        self.type = data.get('type')
        self.date_start = datetime.fromtimestamp(data.get('date_start', 0))
        self.date_end = datetime.fromtimestamp(data.get('date_end', 0))
        self.role_title = data.get('role_title')

    def __str__(self):
        return f"{self.role_title} ({self.date_start.year}-{self.date_end.year})"

class DecidedBy:
    def __init__(self, data):
        self.name = data.get('name')
        self.members = [Justice(j) for j in data.get('members', []) if j]

class WrittenOpinion:
    def __init__(self, data):
        self.id = data.get('id')
        self.title = data.get('title')
        self.author = data.get('author')
        self.type_value = data.get('type', {}).get('value')
        self.type_label = data.get('type', {}).get('label')
        self.justia_opinion_id = data.get('justia_opinion_id')
        self.justia_opinion_url = data.get('justia_opinion_url')
        self.judge_full_name = data.get('judge_full_name')
        self.judge_last_name = data.get('judge_last_name')
        self.title_overwrite = data.get('title_overwrite')
        self.href = data.get('href')

    def __str__(self):
        return f"{self.title} ({self.type_label})"



class Case:
    def __init__(self, data):
        self.id = data.get('ID')
        self.name = data.get('name')
        self.href = data.get('href')
        self.docket_number = data.get('docket_number')
        self.first_party = data.get('first_party')
        self.first_party_label = data.get('first_party_label')
        self.second_party = data.get('second_party')
        self.second_party_label = data.get('second_party_label')
        self.decided_date = datetime.fromtimestamp(data.get('timeline', [{}])[0].get('dates', [0])[0])
        self.citation = Citation(data.get('citation', {}))
        self.advocates = [Advocate(a) for a in data.get('advocates', []) if a] if data.get('advocates') else []
        self.decisions = [Decision(d) for d in data.get('decisions', []) if d] if data.get('decisions') else []
        self.decided_by = DecidedBy(data.get('decided_by', {})) if data.get('decided_by') else None
        self.term = data.get('term')
        self.justia_url = data.get('justia_url')
        self.written_opinion = [WrittenOpinion(o) for o in data.get('written_opinion', []) if o] if data.get('written_opinion') else []


    def __str__(self):
        return f"{self.name} ({self.term})"

    def print_details(self):
        print(f"Case: {self.name}")
        print(f"Href: {self.href}")
        print(f"Docket: {self.docket_number}")
        print(f"Citation: {self.citation}")
        print(f"Decided: {self.decided_date.strftime('%B %d, %Y')}")
        print(f"Parties: {self.first_party} v. {self.second_party}")
        print("\nAdvocates:")
        for advocate in self.advocates:
            print(f"  {advocate}")
        print("\nDecisions:")
        for decision in self.decisions:
            print(f"  {decision}")
            for vote in decision.votes:
                print(f"    {vote}")

In [71]:
from flair.data import Sentence
from flair.nn import Classifier

# Load the NER and relation classifiers
tagger = Classifier.load('ner')
extractor = Classifier.load('relations')

class Entity:
    def __init__(self, text, label):
        self.text = text
        self.label = label
    
    def __str__(self):
        return f"Entity(text='{self.text}', label='{self.label}')"

class Relation:
    def __init__(self, label, head, tail):
        self.label = label
        self.head = head
        self.tail = tail
    
    def __str__(self):
        return f"Relation(label='{self.label}', head={self.head}, tail={self.tail})"



def extract_entities_and_relations(sentence_text):
    if not sentence_text:
        return [], []
    
    sentence = Sentence(sentence_text)
    
    entities = []

    tagger.predict(sentence)
    for entity in sentence.get_labels('ner'):
        entities.append(Entity(entity.data_point.text, entity.value))
    
    extractor.predict(sentence)
    
    relations = []


# Process relations (edges)
    for relation in sentence.get_labels('relation'):
        
        head_text = relation.data_point.first.text
        head_type = relation.data_point.first.get_label('ner').value
        tail_text = relation.data_point.second.text
        tail_type = relation.data_point.second.get_label('ner').value
        
        head_entity = Entity(head_text, head_type)
        tail_entity = Entity(tail_text, tail_type)
        relations.append(Relation(relation.value, head_entity, tail_entity))
            
    return entities, relations


2024-08-19 14:57:26,599 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [10]:
import os
from openai import OpenAI
from pydantic import BaseModel as BaseModel, Field
from enum import Enum
from typing import List, Union, Literal


# Set up OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# # Define the model to use
MODEL = "gpt-4o-2024-08-06"  # Make sure to use a model that supports Structured Outputs

class LegalEntityType(Enum):
    Case = "Case"   
    Court = "Court"
    Jurisdiction = "Jurisdiction"
    Party = "Party"
    Advocate = "Advocate"
    Justice = "Justice"
    Statute = "Statute"
    Situation = "Situation"
    Standard = "Standard"
    Issue = "Issue"
    Reasoning = "Reasoning"
    PolicyArea = "PolicyArea"
    Remedy = "Remedy"
    Outcome = "Outcome"
    Evidence = "Evidence"    
    Motion = "Motion"        
    
class LegalRelationType(Enum):
    follows_precedent = "follows_precedent"
    distinguishes_from = "distinguishes_from"
    overturns = "overturns"
    interprets_statute = "interprets_statute"
    applies_to = "applies_to"
    construes = "construes"
    asserts_jurisdiction_over = "asserts_jurisdiction_over"
    claims_sovereign_immunity_from = "claims_sovereign_immunity_from"
    determines_scope_of = "determines_scope_of"
    applies_test = "applies_test"
    consists_of = "consists_of"
    establishes_standard_for = "establishes_standard_for"
    dissents_on_grounds = "dissents_on_grounds"
    concurs_with = "concurs_with"
    criticizes = "criticizes"
    impacts_policy_on = "impacts_policy_on"
    considers_policy = "considers_policy"
    affects = "affects"
    orders_remedy = "orders_remedy"
    results_in = "results_in"
    provides_relief_to = "provides_relief_to"
    argues_for = "argues_for"
    challenges = "challenges"
    seeks = "seeks"
    interprets_constitutional_provision = "interprets_constitutional_provision"
    violates = "violates"
    upholds_constitutionality_of = "upholds_constitutionality_of"
    admits = "admits"
    supports = "supports"
    rules_on = "rules_on"
    preempts = "preempts"

class LegalEntity(BaseModel):
    type: LegalEntityType
    name: str
    description: str

class LegalTriple(BaseModel):
    subject: LegalEntity
    predicate: LegalRelationType
    object: LegalEntity
    text_reference: str
    explanation: str

class LegalAnalysis(BaseModel):
    triples: List[LegalTriple]    

LEGAL_ONTOLOGY = [
    ("Case", "establishes_principle", "Principle"),
    ("Court", "reasons_that", "Argument"),
    ("Decision", "based_on", "Reasoning"),
    ("Case", "follows_precedent", "Case"),
    ("Case", "distinguishes_from", "Case"),
    ("Case", "overturns", "Case"),
    ("Court", "interprets_statute", "Statute"),
    ("Statute", "applies_to", "Situation"),
    ("Court", "construes", "StatutoryLanguage"),
    ("Court", "asserts_jurisdiction_over", "Matter"),
    ("Entity", "claims_sovereign_immunity_from", "Jurisdiction"),
    ("Court", "determines_scope_of", "Jurisdiction"),
    ("Court", "applies_test", "Test"),
    ("Test", "consists_of", "TestElement"),
    ("Court", "establishes_standard_for", "Issue"),
    ("Justice", "dissents_on_grounds", "Reasoning"),
    ("Justice", "concurs_with", "MajorityOpinion"),
    ("DissentingOpinion", "criticizes", "MajorityOpinion"),
    ("Decision", "impacts_policy_on", "PolicyArea"),
    ("Court", "considers_policy", "PolicyConsideration"),
    ("Ruling", "affects", "SocialOrEconomicOutcome"),
    ("Court", "orders_remedy", "Remedy"),
    ("Decision", "results_in", "Outcome"),
    ("Judgment", "provides_relief_to", "Party"),
    ("Party", "argues_for", "Position"),
    ("Party", "challenges", "Action"),
    ("Plaintiff", "seeks", "Relief"),
    ("Decision", "interprets_constitutional_provision", "ConstitutionalClause"),
    ("Law", "violates", "ConstitutionalRight"),
    ("Court", "upholds_constitutionality_of", "Statute"),
    ("Court", "admits", "Evidence"),
    ("Evidence", "supports", "Claim"),
    ("Court", "rules_on", "ProceduralMotion"),
    ("Party", "files", "Motion"),
    ("Statute", "preempts", "StateLaw")
]

def extract_legal_triples(text: str) -> List[LegalTriple]:
    prompt = f"""
    As an AI assistant specialized in legal analysis, your task is to analyze the given legal text and extract specific entities and their relationships based on the following ontology:

    {LEGAL_ONTOLOGY}

    For each relevant relationship found in the text:
    1. Extract the specific entities that fit the subject and object types from the ontology.
    2. Formulate a triple using these specific entities and the predicate from the ontology.
    3. Include a brief quote or paraphrase from the text supporting the triple.
    4. Offer a short explanation of the triple's significance in the context of the case.

    Only include triples that are directly relevant to the given text. Do not force relationships that are not clearly present in the document.

    Here is the legal text to analyze:

    {text}

    Remember to extract specific entities from the text, not just use generic terms. For each entity, provide its type (e.g., LegalCase, LegalCourt, etc.) and either a name or a description, depending on what's most appropriate for that entity type.
    """

    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a legal analysis expert."},
            {"role": "user", "content": prompt}
        ],
        response_format=LegalAnalysis,
    )

    return response.choices[0].message.parsed.triples

# def process_legal_document(document: str) -> List[LegalTriple]:
#     legal_triples = extract_legal_triples(document)
    
#     entities = set()
#     relations = []

#     for triple in legal_triples:
#         subject_entity = (triple.subject.type, triple.subject.name or triple.subject.description)
#         object_entity = (triple.object.type, triple.object.name or triple.object.description)
        
#         entities.add(subject_entity)
#         entities.add(object_entity)
        
#         relation = (subject_entity, triple.predicate, object_entity)
#         relations.append(relation)
    
#     return list(entities), relations
#     return extract_legal_triples(document)
def process_legal_document(document: str) -> List[LegalTriple]:
    return extract_legal_triples(document)


In [11]:
def process_legal_document(document: str) -> List[LegalTriple]:
    return extract_legal_triples(document)

# Example usage
if __name__ == "__main__":
    sample_text = """
    The Supreme Court ruled in Oklahoma Tax Commission v. Citizen Band Potawatomi Indian Tribe of Oklahoma that under the doctrine of tribal sovereign immunity, 
    Oklahoma may not tax sales to tribal members on tribal land, but the tribe has an obligation to assist in collecting valid state taxes on 
    sales to non-members. The Court cited United States v. United States Fidelity and Guaranty Co. to support its reasoning on tribal sovereign immunity. 
    Justice Stevens concurred, noting the need for Congressional action to address state-tribal tax issues. The Court also established that trust land 
    qualifies as a reservation for tribal immunity purposes when it has been validly set apart for the use of Indians under government superintendence.
    """
    
    legal_triples = process_legal_document(sample_text)
    print(f"Total triples extracted: {len(legal_triples)}")
    for i, triple in enumerate(legal_triples, 1):
        print(f"\nTriple {i}:")
        print(f"Subject: {triple.subject.type} - {triple.subject.name or triple.subject.description}")
        print(f"Predicate: {triple.predicate}")
        print(f"Object: {triple.object.type} - {triple.object.name or triple.object.description}")
        print(f"Text Reference: \"{triple.text_reference}\"")
        print(f"Explanation: {triple.explanation}")

Total triples extracted: 4

Triple 1:
Subject: LegalEntityType.Case - Oklahoma Tax Commission v. Citizen Band Potawatomi Indian Tribe of Oklahoma
Predicate: LegalRelationType.follows_precedent
Object: LegalEntityType.Case - United States v. United States Fidelity and Guaranty Co.
Text Reference: "The Court cited United States v. United States Fidelity and Guaranty Co. to support its reasoning on tribal sovereign immunity."
Explanation: The Supreme Court relied on the precedent set by United States v. United States Fidelity and Guaranty Co. to reinforce its decision regarding tribal sovereign immunity, emphasizing the consistency in legal interpretations of tribal rights over time.

Triple 2:
Subject: LegalEntityType.Court - Supreme Court
Predicate: LegalRelationType.asserts_jurisdiction_over
Object: LegalEntityType.Jurisdiction - Tribal Sovereign Immunity
Text Reference: "The Supreme Court ruled in Oklahoma Tax Commission v. Citizen Band Potawatomi Indian Tribe of Oklahoma that under t

In [12]:

import hashlib
def calculate_hash(text):
    return hashlib.md5(text.encode()).hexdigest()


In [13]:
def legal_triples_to_entities_and_relations(triples: List[LegalTriple]):
    entities = {}
    relations = []

    for triple in triples:
        subject_entity = Entity(triple.subject.name if hasattr(triple.subject, 'name') else str(triple.subject), triple.subject.type.value)
        object_entity = Entity(triple.object.name if hasattr(triple.object, 'name') else str(triple.object), triple.object.type.value)

        # Use hash to avoid duplicate entities
        subject_hash = calculate_hash(subject_entity.text)
        object_hash = calculate_hash(object_entity.text)

        if subject_hash not in entities:
            entities[subject_hash] = subject_entity
        if object_hash not in entities:
            entities[object_hash] = object_entity

        relation = Relation(triple.predicate.value, subject_entity, object_entity)
        relations.append(relation)

    return list(entities.values()), relations

In [14]:
def entity_to_node(entity: Entity):
    return Node(entity.text, entity.label, {"name": entity.text})
    

def relation_to_nodes_and_edges(relation: Relation):
    head_node = entity_to_node(relation.head)
    tail_node = entity_to_node(relation.tail)      
    relation_edge = Edge(head_node.id, tail_node.id, relation.label, {}, "count")    
    return [[head_node, tail_node], relation_edge]



In [15]:
opinions_db = TinyDB('opinions.db.json')

import hashlib

def calculate_hash(text):
    return hashlib.md5(text.encode()).hexdigest()

def process_scotus_opinion(writtenOpinion: WrittenOpinion, case_node: Node):
    opinion = opinions_db.get(where('id') == writtenOpinion.id)
    if not opinion or "content" not in opinion:
        return [], []
    
    legal_triples = process_legal_document(opinion["content"])
    entities, relations = legal_triples_to_entities_and_relations(legal_triples)
    save_opinion_embedding(case_node.id, opinion["content"])
    return entities, relations
        
    
    # sentences = opinion['content'].split('. ')
    
    # all_entities = {}
    # all_relations = {}
    
    # for sentence in sentences:
    #     entities, relations = extract_entities_and_relations(sentence)
        
    #     for entity in entities:
    #         if entity and entity.text:
    #             entity_hash = calculate_hash(entity.text)
    #             if entity_hash not in all_entities:
    #                 all_entities[entity_hash] = entity
        
    #     for relation in relations:
    #         if relation and relation.label:
    #             relation_hash = calculate_hash(relation.label)
    #             if relation_hash not in all_relations:
    #                 all_relations[relation_hash] = relation
    
    # return list(all_entities.values()), list(all_relations.values())

In [16]:
def process_scotus_opinions(written_opinions: list[WrittenOpinion], case_node: Node): 
  if written_opinions:
    for opinion in written_opinions:
      if opinion:
        
        opinion_node = Node(opinion.id, "Opinion", {"title": opinion.title, "case_id": case_node.id})
        opinion_node.create_node()
        case_opinion_edge = Edge(case_node.id, opinion_node.id, "case_opinion", {}, "count")
        case_opinion_edge.create_edge()
        
        entities, relations = process_scotus_opinion(opinion, case_node)
        if entities:
          entities_nodes = [entity_to_node(entity) for entity in entities if entity]
          for relation in relations:
            if relation:
              relation_nodes, relation_edge = relation_to_nodes_and_edges(relation)
              head_node, tail_node = relation_nodes
              if head_node and tail_node:
                head_node.create_node()
                tail_node.create_node()
                relation_edge.create_edge()

          for node in entities_nodes:
            if node:
              node.create_node()
              mentioned_in_edge = Edge(case_node.id, node.id, "mentioned_in", {}, "count")
              mentioned_in_edge.create_edge()

        print(".", end="")

In [17]:
def process_scotus_case(case: Case):
  first_party = case.first_party
  second_party = case.second_party
  advocates = case.advocates if case.advocates else []
  decisions = case.decisions if case.decisions else []
  justices = case.decided_by.members if case.decided_by and case.decided_by.members else []
  
  case_node = Node(case.id, "Case", {"name": case.name, "docket_number": case.docket_number, "term": case.term, "decided_date": case.decided_date.strftime('%Y-%m-%d')})
  case_node.create_node()
  
  if first_party:
      first_party_node = Node(first_party,"Party", {"name": first_party})
      first_party_node.create_node()
      first_party_node_name = case.first_party_label      
      case_party_edge_1 = Edge(case_node.id, first_party_node.id, first_party_node_name, {}, "count")
      case_party_edge_1.create_edge()
  
  if second_party:
      second_party_node = Node(second_party, "Party", {"name": second_party})
      second_party_node.create_node()
      second_party_node_name = case.second_party_label
      case_party_edge_2 = Edge(case_node.id, second_party_node.id, second_party_node_name, {}, "count")
      case_party_edge_2.create_edge()  
  
  for advocate in advocates:
    advocate_node = Node(advocate.name, "Advocate", {"name": advocate.name, "description": advocate.description})
    advocate_node.create_node()
    advocate_edge = Edge(case_node.id, advocate_node.id, "advocated_by", {}, "count")
    advocate_edge.create_edge()
  
  for justice in justices:
    justice_node = Node(justice.id, "Justice", {"name": justice.name})
    justice_node.create_node()
    justice_edge = Edge(case_node.id, justice_node.id, "decided_by", {}, "count")
    justice_edge.create_edge()
    
  for decision in decisions:
    if decision.winning_party:
        decision_node = Node(decision.winning_party, "Party", { "name": decision.winning_party})
        decision_node.create_node()
        decision_edge = Edge(case_node.id, decision_node.id, "won_by", {
          "decision_type": decision.decision_type
        }, "count")
        decision_edge.create_edge()
    for vote in decision.votes:
      justice_node = Node(vote.member.id, "Justice", {"name": vote.member.name})
      justice_node.create_node()
      vote_edge = Edge(case_node.id, justice_node.id, vote.vote, {
        "opinion_type": vote.opinion_type
      }, "count")
      vote_edge.create_edge()
  
  # print(len(case.written_opinion))  
  process_scotus_opinions(case.written_opinion, case_node)

In [20]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [25]:
from ipywidgets import FloatProgress

from tqdm.notebook import tqdm
import random

db = TinyDB('cases.db.json')

expanded_cases = db.all()


sampled_cases = random.sample(expanded_cases, 100)



for case_data in tqdm(sampled_cases):
    case = Case(case_data)
    # case.print_details()
    

    # print("\n")
    process_scotus_case(case)

  0%|          | 0/100 [00:00<?, ?it/s]

...............................................................................................................................................................................................................................................................................

TypeError: 'NoneType' object is not iterable

In [ ]:
import os

env_vars_to_delete = [
    "OPENAI_API_KEY",
    "PINECONE_API_KEY",
    "NEO4J_URI",
    "NEO4J_USERNAME",
    "NEO4J_PASSWORD"
]


for var in env_vars_to_delete:
    if var in os.environ:
        del os.environ[var]

